# Notebook for early fault detection for WT04

## Table of contents
1. Raw signal
2. Resampled signal
3. Filtered signal with optimal bandpass based on SK
4. Demodulasation using EEMD signals
5. Selecting IMF: Correlation coefficient
6. Envelope spectral analyis
7. Frequency study

## 1. Raw signal 

## 2. Resampled signal

## 3. Filtered signal with optimal bandpass based on SK

## 4. Demodulasation using EEMD signals

## 5. Selecting IMF: Correlation coefficient

## 6. Envelope spectral analysis

## 7. Frequency study (fault characteristics), fault development